In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import os

In [4]:
import ee
import geemap

Map = geemap.Map()
ee.Authenticate()
ee.Initialize()

# s2data = require('users/georgeazzari/EEtools:s2.data.js')
# fsetrees = require('users/georgeazzari/EEtools:s2.cloudtree.fse.africa.js')

def decodeQA60(img):
    qa60 = img.select('QA60')#.updateMask(img.select('B2'))
    cloudBitMask = qa60.bitwiseAnd(ee.Number(2).pow(10).int())
    cirrusBitMask = qa60.bitwiseAnd(ee.Number(2).pow(11).int())
    clear = cloudBitMask.eq(0).And(cirrusBitMask.eq(0)).rename(['PXQA60_CLEAR']).toInt()
    clear = clear.updateMask(clear)
    return img.addBands([clear])

def applyCloudmask(img):
    clearmask = img.select('PXQA60_CLEAR')
    return img.updateMask(clearmask)



Enter verification code: 4/1AX4XfWjihEMT7yZAV8dbS9lCatYCXem_TCf7_j2h6hJTlXfU_pgO-dEZMeI

Successfully saved authorization token.


Functions for mosaic sentienl-2 imagery

In [6]:
def cutoverlap(img): 
    geo1 = img.geometry();
    geo2 = img.translate(-9900.0, -9900, 'meters').geometry();
    geo = geo1.intersection(geo2)
    return ee.Algorithms.If(geo.geometries().length(), img.clip(geo), None);

def get_orbit_num(imgCol):
    return imgCol.aggregate_array('SENSING_ORBIT_NUMBER').distinct() ## revised by ZJ

def cloudfree(date):
    aoi = ee.FeatureCollection("TIGER/2018/States").filterMetadata('NAME', 'equals', 'Iowa').geometry()
    aoi = aoi.bounds()
    increase = ee.Number(date)
    start = ee.Date('2018-04-01').advance(increase.multiply(5), 'day')
    end = ee.Date('2018-04-06').advance(increase.multiply(5), 'day')
    s2col = ee.ImageCollection("COPERNICUS/S2").filterDate(start, end).filterBounds(aoi).map(cutoverlap, True)\
.filterMetadata('MGRS_TILE', 'equals', '14TQM')
    orbit = get_orbit_num(s2col)
    def mosaic(i):
        ds = s2col.filter(ee.Filter.eq('SENSING_ORBIT_NUMBER', i))
        return ds.mosaic()  ## revised by ZJ
    mosaiced = orbit.map(mosaic)
    cloud_mask =  ee.ImageCollection(mosaiced).map(decodeQA60).map(applyCloudmask)
    return cloud_mask.mosaic()  ##revised by ZJ
#     return ee.ImageCollection(mosaiced).mosaic()

In [7]:
def build_from_corner(feature, dx, dy, projection=None):

    coordinates = feature.geometry().coordinates()

    leftX = ee.Number(coordinates.get(0))
    leftY = ee.Number(coordinates.get(1))
    rightX = ee.Number(coordinates.get(0)).add(dx)
    rightY = ee.Number(coordinates.get(1)).add(dy)
    corners = ee.List([leftX, leftY, rightX, rightY])
    
    if projection is None:
        geometry = ee.Geometry.Rectangle(corners)
    else:
        geometry = ee.Geometry.Rectangle(corners, proj=projection)

    return ee.Feature(geometry)

def get_box_grid(box, nx, ny):

    coords = ee.List(box.coordinates())
    minlon = ee.Number(ee.List(ee.List(coords.get(0)).get(0)).get(0))
    maxlon = ee.Number(ee.List(ee.List(coords.get(0)).get(2)).get(0))
    minlat = ee.Number(ee.List(ee.List(coords.get(0)).get(0)).get(1))
    maxlat = ee.Number(ee.List(ee.List(coords.get(0)).get(2)).get(1))

    dx = maxlon.subtract(minlon).abs().divide(nx-1)
    dy = maxlat.subtract(minlat).abs().divide(ny-1)

    lons = ee.List.sequence(minlon, maxlon, None, nx).slice(0, -1)
    lats = ee.List.sequence(minlat, maxlat, None, ny).slice(0, -1)

    def outter_map(x):
        def inner_map(y):
            return ee.Feature(ee.Geometry.Point([ee.Number(x), ee.Number(y)]))
        return lats.map(inner_map)

    pcoords = lons.map(outter_map)

    ptsgrid = ee.FeatureCollection(pcoords.flatten())
    grid = ptsgrid.map(lambda p: build_from_corner(p, dx, dy).set('corner', p))

    return grid

In [11]:
# grid = get_box_grid(aoi, 10, 10)
aoi = ee.FeatureCollection("TIGER/2018/States").filterMetadata('NAME', 'equals', 'Iowa').geometry()
CDL = ee.ImageCollection("USDA/NASS/CDL").filterDate('2017-01-01', '2017-12-31')
fr = ee.Image("users/lin00370/france/RPG2019_2").geometry()

In [9]:
for i in range(0, 24):
    intervals = ee.List.sequence(i, i)
    countCol = intervals.map(cloudfree)
    Map.addLayer(ee.Image(countCol.get(0)))
#     band = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
    band = ['B5', 'B11']
    geemap.ee_export_image_to_drive(ee.Image(countCol.get(0)).select(band), \
                         description="ia_2018_" + str(i), \
                         folder='GEE_IA_2018', \
                         max_pixels=1e13, \
                         region=ee.ImageCollection("COPERNICUS/S2").filterMetadata('MGRS_TILE', 'equals', '14TQM').first().geometry().bounds(), \
                         crs='EPSG:4326', \
                         scale=30)
    Map

Exporting ia_2018_0 ...
Exporting ia_2018_1 ...
Exporting ia_2018_2 ...
Exporting ia_2018_3 ...
Exporting ia_2018_4 ...
Exporting ia_2018_5 ...
Exporting ia_2018_6 ...
Exporting ia_2018_7 ...
Exporting ia_2018_8 ...
Exporting ia_2018_9 ...
Exporting ia_2018_10 ...
Exporting ia_2018_11 ...
Exporting ia_2018_12 ...
Exporting ia_2018_13 ...
Exporting ia_2018_14 ...
Exporting ia_2018_15 ...
Exporting ia_2018_16 ...
Exporting ia_2018_17 ...
Exporting ia_2018_18 ...
Exporting ia_2018_19 ...
Exporting ia_2018_20 ...
Exporting ia_2018_21 ...
Exporting ia_2018_22 ...
Exporting ia_2018_23 ...


In [12]:
geemap.ee_export_image_to_drive(ee.Image(CDL.first()).select('cropland'), \
                     description="IA_CDL_2017", \
                     folder='GEE_IA_2018', \
                     max_pixels=1e13, \
                     region=ee.ImageCollection("COPERNICUS/S2").filterMetadata('MGRS_TILE', 'equals', '14TQM').first().geometry().bounds(), \
#                      region=mc, \
                     crs='EPSG:4326', \
                     scale=30)
# arr = geemap.ee_to_numpy(ee.Image(test))

Exporting IA_CDL_2017 ...
